# Time series denoising

This notebook is the Python version of the exercises of [Signal processing problems](https://www.udemy.com/course/signal-processing/). 

## Running mean time series

A running mean time series is a filter that can be applied to smooth a given signal. For this case, a given point $y_{t}$ is equal to the average of its neighbour points. This can be expressed by:

$y_{1} = \dfrac{1}{2k+1} \sum_{i=t-k}^{t+k}x_{i}$.

This filter smoothes the sharp edges and the larger the value of k, the smoother the signal becomes. Unfortunately, this produces *edge effects* for the first and last signal points. 

It's generally appropriate when the noise is evenly (positively and negatively) distributed.

In [9]:
## Importing the libraries
import numpy as np
from scipy.interpolate import interp1d

In [12]:
## Setting up variables
sampling_rate = 1000
time = np.arange(0, 3, 1/sampling_rate) # Step size is 1/sampling_rate (time interval between samples)
n = len(time)
p = 15  # Poles for random interpolation
noiseamp = 5  # Noise level in standard deviations

## Amplitude modulator and noise level

# Creates an array of p random variables, between 0 and 1, scaling by 30
# Represents random amplitude values (y-axis values)
random_ampl_values = np.random.rand(p) * 30

# Generates p equally spaced values from 0 to p-1
# Represents the time/index points (x-axis values)
interp_func = interp1d(np.linspace(0, p-1, p), random_ampl_values, kind='linear')

# Generates n points to match teh samples of time vector
# Uses linear interpolation to fill in these values
ampl = interp_func(np.linspace(0, p-1, n))

# Generates n samples from a std normal distribution and scales the noise
noise = noiseamp * np.random.randn(n)

# Our signal is the sum of the amplitude and the noise
signal = ampl + noise

# Initializing the filtered signal that has the same shape and data type of signal
filtered_signal = np.zeros_like(signal)
